<a href="https://colab.research.google.com/github/pmb06d/zero_rated_spots/blob/master/P%26G_Zero_Rated_Spots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set up the Google Drive Client

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import os

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

# This ID below is the code for the folder
file_list = drive.ListFile(
    {'q': "'1FbRkOGDnGfeAxIgAI-rY7L-d75nQDI2W' in parents"}).GetList()


In [0]:
# Get a list of the file names for display

titles = [file['title'] for file in file_list]
print('There are',str(len(titles)),'files')

There are 625 files


In [0]:
# we need a list of IDs

# The shareable link

import pandas as pd

id_list = [file['id'] for file in file_list]

temp = pd.DataFrame(list(zip(id_list,titles)))

accept = []
reject = []

for elem in titles:
    if str(elem).endswith('.txt'):
        accept.append(elem)
    else:
        reject.append(elem)
        
id_list = list(temp.loc[temp[1].isin(accept),0])

In [0]:
import pandas as pd
import time

start_time = time.time()

df_list = []

for id in id_list:
  # Storing the file into a dataframe
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('Filename.csv')  
  df = pd.read_csv('Filename.csv', sep = ';',encoding='latin1' )
  df_list.append(df)
  
print("--- %s seconds ---" % (time.time() - start_time),'\n')

--- 420.6923232078552 seconds --- 



In [0]:
full_df = pd.concat(df_list, ignore_index=True)

full_df.shape

(36600000, 7)

In [0]:
list(full_df)

[' ', 'Region', 'Target', 'Channel', 'Date', 'TimeBand', 'Rat% {Av(Wg)}']

In [0]:
# They are all strings we need to clean this
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36600000 entries, 0 to 36599999
Data columns (total 7 columns):
                 int64
Region           object
Target           object
Channel          object
Date             object
TimeBand         object
Rat% {Av(Wg)}    float64
dtypes: float64(1), int64(1), object(5)
memory usage: 1.9+ GB


In [0]:
full_df.head()

,,Region,Target,Channel,Date,TimeBand,Rat% {Av(Wg)}
0,1,PAN,Pay-Subs&Comm_P18-49,Cinemax,"Mon Jan 01, 2018",06:00:00 - 07:00:00,0.048590
1,2,PAN,Pay-Subs&Comm_P18-49,Cinemax,"Mon Jan 01, 2018",07:00:00 - 08:00:00,0.039990
2,3,PAN,Pay-Subs&Comm_P18-49,Cinemax,"Mon Jan 01, 2018",08:00:00 - 09:00:00,0.038025
3,4,PAN,Pay-Subs&Comm_P18-49,Cinemax,"Mon Jan 01, 2018",09:00:00 - 10:00:00,0.065623
4,5,PAN,Pay-Subs&Comm_P18-49,Cinemax,"Mon Jan 01, 2018",10:00:00 - 11:00:00,0.042717


In [0]:
def get_start(timeband):
  temp_list = timeband.replace(' ','').split('-')
  start_time = temp_list[0][:5]
  return(start_time)

def get_label(timeband):
  temp_list = timeband.replace(' ','').split('-')

  start_time = temp_list[0].split(':')
  start_time = (int(start_time[0])*60)+int(start_time[1])

  end_time = temp_list[1].split(':')
  end_time = (int(end_time[0])*60)+int(end_time[1])

  label = str(int(end_time-start_time))+' minute(s)'
  return(label)

# add the custom timebands

def identify_cinemaxtb(timeband):
      temp_list = timeband.replace(' ','').split('-')
      start_hour = int(temp_list[0][:1])

      if start_hour < 13:
        return('Morning')
      elif start_hour < 16:
        return('Afternoon')
      elif start_hour < 26:
        return('Primetime')
      else:
        return('Late')

In [0]:
full_df['Start time'] = full_df['TimeBand'].apply(get_start)
full_df['Interval'] = full_df['TimeBand'].apply(get_label)
full_df['ROS TimeBands'] = full_df['TimeBand'].apply(identify_cinemaxtb)

full_df.head()

,,Region,Target,Channel,Date,TimeBand,Rat% {Av(Wg)},Start time,Interval,ROS TimeBands
0,1,PAN,Pay-Subs&Comm_P18-49,Cinemax,"Mon Jan 01, 2018",06:00:00 - 07:00:00,0.048590,06:00,60 minute(s),Morning
1,2,PAN,Pay-Subs&Comm_P18-49,Cinemax,"Mon Jan 01, 2018",07:00:00 - 08:00:00,0.039990,07:00,60 minute(s),Morning
2,3,PAN,Pay-Subs&Comm_P18-49,Cinemax,"Mon Jan 01, 2018",08:00:00 - 09:00:00,0.038025,08:00,60 minute(s),Morning
3,4,PAN,Pay-Subs&Comm_P18-49,Cinemax,"Mon Jan 01, 2018",09:00:00 - 10:00:00,0.065623,09:00,60 minute(s),Morning
4,5,PAN,Pay-Subs&Comm_P18-49,Cinemax,"Mon Jan 01, 2018",10:00:00 - 11:00:00,0.042717,10:00,60 minute(s),Morning


In [0]:
full_df.shape

# this should say 10 columns

(36600000, 10)

In [0]:
from google.colab import files

# e.g. save pandas output as csv
full_df.to_csv('P&G Mexico Analysis.csv', index=False)

files.download('P&G Mexico Analysis.csv')

In [0]:
full_df.to_csv('P&G Mexico Analysis.csv', index=False)

drive_service = build('drive', 'v3')

def save_file_to_drive(name, path):
    file_metadata = {'name': name, 'mimeType': 'application/octet-stream'}
      
    media = MediaFileUpload(path, mimetype='application/octet-stream', resumable=True)

    created = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

    print('File ID: {}'.format(created.get('id')))

    return created

In [0]:
save_file_to_drive('P&G Mexico Analysis.csv', 'P&G Mexico Analysis.csv')

File ID: 1AaLzcR7I9PNrfILxCoDdZouDXSd8P7wM


{'id': '1AaLzcR7I9PNrfILxCoDdZouDXSd8P7wM'}